In [ ]:
%ls

In [ ]:
!chmod 777 -R *.sh *.py *.pl

In [ ]:
%ls

In [ ]:
!bash DB1_NCBI_download_refs.sh -h 

In [ ]:
%%bash
./NCBI_download_refs.sh -n Streptophyta -t txid35493 -m trnL

In [ ]:
%ls

In [ ]:
!bash DB2_primers_cut.sh -h

In [ ]:
%%bash
./DB2_primers_cut.sh -r Streptophyta_trnL_NCBI.fasta -F GGGCAATCCTGAGCCAA -R CCATTGAGTCTCTGCACCTATC -p target_trnL

In [ ]:
%ls -t | head

In [ ]:
!head target_trnL_cutted_references.fasta

In [ ]:
!grep -A 20 "MT950540.1" Streptophyta_trnL_NCBI.fasta

In [ ]:
!bash DB3_NCBI_lineage.sh -h

In [ ]:
%%bash
./DB3_NCBI_lineage.sh -r target_trnL_cutted_references.fasta -p target_trnL

In [ ]:
!tail target_trnL.log

In [ ]:
!head target_trnL.uniq.fasta

In [ ]:
%ls | head

In [ ]:
!bash 1_merge_pear.sh -h

In [7]:
%%bash
./1_merge_pear.sh -f trnL_S1_L001_R1_001.fastq -r trnL_S1_L001_R2_001.fastq -o trnL -p 0.001 -s 20 -t 4

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.6 [January 15, 2015]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: trnL_S1_L001_R1_001.fastq
Reverse reads file.................: trnL_S1_L001_R2_001.fastq
PHRED..............................: 33
Using empirical frequencies........: YES
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.001000
Quality score threshold (trimming).: 20
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 0.000000
Minimum overlap....................: 20
Scoring method.....................: Scaled score
Threads............................: 4

Allocating memory..................: 200,0

In [8]:
%ls -lat | head

total 6585452
-rw-r--r--  1 jupyter jupyter 1196781550 Dec  8 18:04 trnL.assembled.fastq
-rw-r--r--  1 jupyter jupyter    4076508 Dec  8 18:04 trnL.discarded.fastq
drwxr-xr-x  9 jupyter jupyter     106496 Dec  8 17:58 ./
-rwxrwxrwx  1 jupyter jupyter      15671 Dec  8 17:58 metapipe-run.ipynb*
-rw-r--r--  1 jupyter jupyter          0 Dec  8 17:56 trnL.unassembled.forward.fastq
-rw-r--r--  1 jupyter jupyter          0 Dec  8 17:56 trnL.unassembled.reverse.fastq
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 17:55 Barcodes_LA1.fa
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 17:55 Barcodes_LA2.fa
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 17:55 Barcodes_LA3.fa


In [9]:
!head trnL.assembled.fastq

@M02980:420:000000000-G84WN:1:1101:13782:1799 1:N:0:TTCTACAC+GCCAAGGT
ATGTCCGACCAAGGGCAATCCTGAGCCAAATCCTGTTTTCCGAAAACAAACACAAAGATTCAGAAAGCAAGAATAAAAAAGGATAGGTGCAGAAACTCAATGGGCATATAGT
+
II?IIIIIIIIIII@IIIIIIIAI@IIIIIIIIIIIIIII?IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIBG
@M02980:420:000000000-G84WN:1:1101:16730:1804 1:N:0:GTCTACAC+GCCAAGGT
CCATCACATAGGCCATTGAGTCTCTGCACCTATCCTTTTTTATTCTCGGTTTATGAAACCCTTGTTTGTTTTAATAAAACAGGATTTGGCTCAGGATTGCCCTTGCCACTAG
+
III>BIIIIIIIIIIIIIIIIIIIICIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIAAIIIIIIIIHHIHIIIIIIIIIIIIIIIAIIGFI@IIII@IIIIIIIIIIGIA
@M02980:420:000000000-G84WN:1:1101:13325:1809 1:N:0:TTCTACAC+GCCAAGGT
TGTCTCGCAAGCCCATTGAGTCTCTGCACCTATCCTTTTTTATTCTTGCTTTCTGAATCTTTGTGTTTGTTTTCGGAAAACAGGATTTGGCTCAGGATTGCCCAGCCCTAACTAT


In [10]:
!tail trnL_sub_tags.txt

Peoniasuffruticosa_a	CCATCACATAGG	TGCAGATCCAAC
Peoniasuffruticosa_b	GTGGTATGGGAG	TGCAGATCCAAC
Poriacocos_a	ATAGTTAGGGCT	CCTAAACTACGG
Poriacocos_b	GCACTTCATTTC	CCTAAACTACGG
Prunuspersica_a	ATGTCCGACCAA	TGCAGATCCAAC
Prunuspersica_b	ATCTAGTGGCAA	CCATCACATAGG
Trogopterusxanthipes_a	ATAGTTAGGGCT	TGTTGCGTTTCT
Trogopterusxanthipes_b	ACTTTAAGGGTG	CACCTTACCTTA
Negative	TGTTGCGTTTCT	GAGACTATATGC
NegativePCR	ATGTCCGACCAA	GAGACTATATGC


In [11]:
%%bash 
perl 2a_preparing_tags_LCPI.pl trnL_sub_tags.txt linked

In [12]:
%ls -lat | head

total 6585452
drwxr-xr-x  9 jupyter jupyter     106496 Dec  8 18:05 ./
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 18:05 Barcodes_LA1.fa
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 18:05 Barcodes_LA2.fa
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 18:05 Barcodes_LA3.fa
-rw-r--r--  1 jupyter jupyter 1196781550 Dec  8 18:04 trnL.assembled.fastq
-rw-r--r--  1 jupyter jupyter    4076508 Dec  8 18:04 trnL.discarded.fastq
-rwxrwxrwx  1 jupyter jupyter      15671 Dec  8 17:58 metapipe-run.ipynb*
-rw-r--r--  1 jupyter jupyter          0 Dec  8 17:56 trnL.unassembled.forward.fastq
-rw-r--r--  1 jupyter jupyter          0 Dec  8 17:56 trnL.unassembled.reverse.fastq


In [13]:
!tail Barcodes_LA1.fa

>Prunuspersica_b
^ATCTAGTGGCAA...CCATCACATAGG$
>Trogopterusxanthipes_a
^ATAGTTAGGGCT...TGTTGCGTTTCT$
>Trogopterusxanthipes_b
^ACTTTAAGGGTG...CACCTTACCTTA$
>Negative
^TGTTGCGTTTCT...GAGACTATATGC$
>NegativePCR
^ATGTCCGACCAA...GAGACTATATGC$


In [26]:
%%bash
./2b_demulti_dual_index_linked.sh trnL_subsample.assembled.fastq

./unknown_LA.fq


In [27]:
%ls -lat | head

total 6782864
drwxr-xr-x 10 jupyter jupyter     106496 Dec  8 19:54 ./
drwxr-xr-x  2 jupyter jupyter       4096 Dec  8 19:54 demulti_linked_samples/
-rwxrwxrwx  1 jupyter jupyter      23765 Dec  8 19:53 metapipe-run.ipynb*
-rw-r--r--  1 jupyter jupyter  202130197 Dec  8 19:51 trnL_subsample.assembled.fastq
-rw-r--r--  1 jupyter jupyter       1959 Dec  8 19:47 demulti_count_trnL.txt
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 18:05 Barcodes_LA1.fa
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 18:05 Barcodes_LA2.fa
-rw-r--r--  1 jupyter jupyter       1653 Dec  8 18:05 Barcodes_LA3.fa
-rw-r--r--  1 jupyter jupyter 1196781550 Dec  8 18:04 trnL.assembled.fastq


In [28]:
%ls demulti_linked_samples/ | head

Angelicaesinensis_a_LA.fq
Angelicaesinensis_b_LA.fq
Carthamustinctorius_a_LA.fq
Carthamustinctorius_b_LA.fq
Citrusaurantium_a_LA.fq
Citrusaurantium_b_LA.fq
Cyperusrotundus_a_LA.fq
Cyperusrotundus_b_LA.fq
GXZYT_a_LA.fq
GXZYT_b_LA.fq


In [29]:
!head demulti_linked_samples/Angelicaesinensis_a_LA.fq

@M02980:420:000000000-G84WN:1:1101:14746:1924 1:N:0:GTCTACAC+GCCAAGGT
caccttaccttaCCATTGAGTCTCTGCACCTATCCCCTTCCTATCTTAGGAGAAGAAACATTGTCTTCATGAACCGGATTTGGCTCAGGATTGCCCcctatgtgatgg
+
IHIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@M02980:420:000000000-G84WN:1:1101:13409:1962 1:N:0:GTCTACAC+GCCAAGGT
caccttaccttaGGGCAATCCTGAGCCAAATCCTATTTTCCAAAAACAAACAAAGGCCCAGAAGGTGAAAAAAGGATAGGTGCAGAGACTCAATGGcctatgtgatgg
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@M02980:420:000000000-G84WN:1:1101:16331:1965 1:N:0:GTCTACAC+GCCAAGGT
caccttaccttaGGGCAATCCTGAGCCAAATCCTATTTTCCAAAAACAAACAAAGGCCCAGAAGGTGAAAAAAGGATAGGTGCAGAGACTCAATGGcctatgtgatgg


In [30]:
%%bash
./count_fastq_sequences.sh demulti_linked_samples/*.fq > demulti_count_trnL.txt

In [32]:
!head demulti_count_trnL.txt

	demulti_linked_samples/Angelicaesinensis_a_LA.fq : 22,358
	demulti_linked_samples/Angelicaesinensis_b_LA.fq : 31,246
	demulti_linked_samples/Carthamustinctorius_a_LA.fq : 29,172
	demulti_linked_samples/Carthamustinctorius_b_LA.fq : 11,424
	demulti_linked_samples/Citrusaurantium_a_LA.fq : 2,876
	demulti_linked_samples/Citrusaurantium_b_LA.fq : 56,024
	demulti_linked_samples/Cyperusrotundus_a_LA.fq : 32,552
	demulti_linked_samples/Cyperusrotundus_b_LA.fq : 2,299
	demulti_linked_samples/GXZYT_a_LA.fq : 37,335
	demulti_linked_samples/GXZYT_b_LA.fq : 2,810


In [33]:
!bash 3_tag_primer_clipping_clean_derep.sh -h

##################################################
TAGS AND PRIMERS CLIPPING
##################################################
get your primer's reverse complement in this website:
http://arep.med.harvard.edu/labgc/adnan/projects/Utilities/revcomp.html
 
Usage: 3_tag_primer_clipping_clean_derep.sh [-b tags file] [-F primer forwad] [-R primer reverse] [-f primer forward] [-r primer reverse] [-p prefix] [-l amplicon min length] [-d path to samples]
-b     original barcodes/tags file
-F     primer forward
-R     primer reverse
-f     reverse complement primer forward
-r     everse complement primer reverse
-p     prefix to quality file
-l     minimum length for clipped amplicon
-d     path to demultiplexed samples directory. Use pwd.
-h     print this help
 
##################################################


In [36]:
%%bash
./3_tag_primer_clipping_clean_derep.sh -b trnL_sub_tags.txt -F GGGCAATCCTGAGCCAA -R CCATTGAGTCTCTGCACCTATC \
-f TTGGCTCAGGATTGCCC -r GATAGGTGCAGAGACTCAATGG -p trnL_sub -l 30 -d /home/jupyter/METAPIPE-Notebooks/demulti_linked_samples

minseqlength 32: 12 sequences discarded.
minseqlength 32: 2 sequences discarded.
minseqlength 32: 2 sequences discarded.
mkdir: cannot create directory ‘dereplicated’: File exists


In [37]:
%ls -lat | head

total 3923496
drwxr-xr-x  8 jupyter jupyter       4096 Dec  8 20:03 .git/
drwxr-xr-x  9 jupyter jupyter     106496 Dec  8 20:03 ./
-rwxrwxrwx  1 jupyter jupyter      24252 Dec  8 20:03 metapipe-run.ipynb*
drwxr-xr-x  2 jupyter jupyter       4096 Dec  8 20:02 dereplicated/
-rw-r--r--  1 jupyter jupyter    1297650 Dec  8 20:02 trnL_sub.qual
drwxr-xr-x  5 jupyter jupyter       4096 Dec  8 20:02 clip_out/
-rw-r--r--  1 jupyter jupyter       1909 Dec  8 19:55 demulti_count_trnL.txt
drwxr-xr-x  2 jupyter jupyter       4096 Dec  8 19:54 demulti_linked_samples/
-rw-r--r--  1 jupyter jupyter  202130197 Dec  8 19:51 trnL_subsample.assembled.fastq


In [38]:
%ls clip_out/adapter_clip | head

Angelicaesinensis_a_trim1.fq
Angelicaesinensis_b_trim1.fq
Carthamustinctorius_a_trim1.fq
Carthamustinctorius_b_trim1.fq
Citrusaurantium_a_trim1.fq
Citrusaurantium_b_trim1.fq
Cyperusrotundus_a_trim1.fq
Cyperusrotundus_b_trim1.fq
GXZYT_a_trim1.fq
GXZYT_b_trim1.fq


In [41]:
!head -2 clip_out/adapter_clip/Angelicaesinensis_a_trim1.fq

@M02980:420:000000000-G84WN:1:1101:14746:1924 1:N:0:GTCTACAC+GCCAAGGT
CCATTGAGTCTCTGCACCTATCCCCTTCCTATCTTAGGAGAAGAAACATTGTCTTCATGAACCGGATTTGGCTCAGGATTGCCC


In [42]:
%ls clip_out/primer_clip | head

Angelicaesinensis_a_trim2.fq
Angelicaesinensis_b_trim2.fq
Carthamustinctorius_a_trim2.fq
Carthamustinctorius_b_trim2.fq
Citrusaurantium_a_trim2.fq
Citrusaurantium_b_trim2.fq
Cyperusrotundus_a_trim2.fq
Cyperusrotundus_b_trim2.fq
GXZYT_a_trim2.fq
GXZYT_b_trim2.fq


In [43]:
!head -2 clip_out/primer_clip/Angelicaesinensis_a_trim2.fq

@M02980:420:000000000-G84WN:1:1101:14746:1924 1:N:0:GTCTACAC+GCCAAGGT
CCCTTCCTATCTTAGGAGAAGAAACATTGTCTTCATGAACCGGAT


In [44]:
!grep -A 1 "@M02980:420:000000000-G84WN:1:1101:14746:1924 1:N:0:GTCTACAC+GCCAAGGT" trnL_subsample.assembled.fastq

@M02980:420:000000000-G84WN:1:1101:14746:1924 1:N:0:GTCTACAC+GCCAAGGT
caccttaccttaCCATTGAGTCTCTGCACCTATCCCCTTCCTATCTTAGGAGAAGAAACATTGTCTTCATGAACCGGATTTGGCTCAGGATTGCCCcctatgtgatgg


<p>We can see here the tags in lowercase.&nbsp;</p>
<p>As you can see in the mapping file, the 5&apos; tag is in forward direction <strong>caccttacctta</strong>, and the 3&apos; tag is the reverse complement of the reverse tag <strong>cctatgtgatgg&nbsp;</strong>(prom R2 pair-end read), configuring the tags orientation found in the <strong>Barcodes_LA3.fa</strong> file, which is the most common tag orientation in <strong>merged Illumina paired-end reads</strong>.</p>
<p>Right after the tags in lowercase, we can see the the <strong>3&apos; primer in forward direction</strong> CCATTGAGTCTCTGCACCTATC, the <strong>amplicon </strong>and the <strong>5&apos; primer in reverse direction</strong>.</p>

In [45]:
!head trnL_sub_tags.txt

Angelicaesinensis_a	CACCTTACCTTA	CCATCACATAGG
Angelicaesinensis_b	ACTTTAAGGGTG	CCATCACATAGG
Carthamustinctorius_a	ACTTTAAGGGTG	GAGCAACATCCT
Carthamustinctorius_b	TGTTGCGTTTCT	GAGCAACATCCT
Citrusaurantium_a	GAGACTATATGC	GAGCAACATCCT
Citrusaurantium_b	ATCTAGTGGCAA	TGTTGCGTTTCT
Cyperusrotundus_a	ATAGTTAGGGCT	ACTTTAAGGGTG
Cyperusrotundus_b	GAGACTATATGC	ACTTTAAGGGTG
Glycyrrhizauralensis_a	CCAGGGACTTCT	ACTTTAAGGGTG
Glycyrrhizauralensis_b	ACTTTAAGGGTG	GAGACTATATGC


In [46]:
%ls clip_out/trimming | head

Angelicaesinensis_a_trim3.fq
Angelicaesinensis_b_trim3.fq
Carthamustinctorius_a_trim3.fq
Carthamustinctorius_b_trim3.fq
Citrusaurantium_a_trim3.fq
Citrusaurantium_b_trim3.fq
Cyperusrotundus_a_trim3.fq
Cyperusrotundus_b_trim3.fq
GXZYT_a_trim3.fq
GXZYT_b_trim3.fq


In [47]:
!head -2 clip_out/trimming/Angelicaesinensis_a_trim3.fq

@908d158348fa9fcf1092d4d9f453719cff87c519;ee=0.0045
CCCTTCCTATCTTAGGAGAAGAAACATTGTCTTCATGAACCGGAT


In [48]:
%ls dereplicated | head

Angelicaesinensis_a_dp.fasta
Angelicaesinensis_b_dp.fasta
Carthamustinctorius_a_dp.fasta
Carthamustinctorius_b_dp.fasta
Citrusaurantium_a_dp.fasta
Citrusaurantium_b_dp.fasta
Cyperusrotundus_a_dp.fasta
Cyperusrotundus_b_dp.fasta
GXZYT_a_dp.fasta
GXZYT_b_dp.fasta


In [49]:
!head -2 dereplicated/Angelicaesinensis_a_dp.fasta

>addff39eca064dd9caa294721e6a23cab9c331ab;size=11061
ATCCTATTTTCCAAAAACAAACAAAGGCCCAGAAGGTGAAAAAAG


In [52]:
!head trnL_sub.qual

81113ed4b6436575ac7a53723e4b76ca5e4aab73	0.0030	30
90008a380e5c101173ae114dd518dc14041e6eff	0.0031	31
c32221ff8256031b77747a5c4bb7814d5d981ca5	0.0031	31
e9cec6dc24b78ebea5681e388bb1eeeaa56bd805	0.0031	31
ee1b8a77443ba3fdba9903c5ba69beb30799261b	0.0031	31
054b039ba461a4c6159a15ac2e9777d94e7950b6	0.0032	32
09931875b1c188b228554aafc57231a139dda046	0.0032	32
0c0e27fbec42cd7913536ca330a09777b946878a	0.0359	32
10be97dc5e36afd3f69fb01653755f1dad6c1a3a	0.0032	32
1284bdb3a0f40e0da2696202659ef34a3647f0d5	0.0032	32


In [51]:
%%bash
./4_clustering_swarm_with_chim_filter.sh trnL /home/jupyter/METAPIPE-Notebooks/dereplicated

vsearch v2.18.0_linux_x86_64, 14.7GB RAM, 4 cores
https://github.com/torognes/vsearch

Dereplicating file ./tmp.nR0aPSmsoh 100%
2042036 nt in 34521 seqs, min 32, max 218, avg 59
Sorting 100%
25373 unique sequences, avg cluster 25.4, median 1, max 44856
Writing output file 100%
Swarm 3.0.0
Copyright (C) 2012-2019 Torbjorn Rognes and Frederic Mahe
https://github.com/torognes/swarm

Mahe F, Rognes T, Quince C, de Vargas C, Dunthorn M (2014)
Swarm: robust and fast clustering method for amplicon-based studies
PeerJ 2:e593 https://doi.org/10.7717/peerj.593

Mahe F, Rognes T, Quince C, de Vargas C, Dunthorn M (2015)
Swarm v2: highly-scalable and high-resolution amplicon clustering
PeerJ 3:e1420 https://doi.org/10.7717/peerj.1420

CPU features:      mmx sse sse2 sse3 ssse3 sse4.1 sse4.2 popcnt avx avx2
Database file:     -
Output file:       trnL_cluster_tmp.swarms
Statistics file:   trnL_cluster_tmp.stats
Resolution (d):    1
Threads:           8
Break OTUs:        Yes
Fastidious:        Yes,

In [53]:
%ls -lat | head

total 3928880
drwxr-xr-x 10 jupyter jupyter     106496 Dec  8 20:32 ./
drwxr-xr-x  2 jupyter jupyter       4096 Dec  8 20:32 chimera_out/
-rw-r--r--  1 jupyter jupyter     435371 Dec  8 20:32 trnL_cluster_representatives.uchime
-rw-r--r--  1 jupyter jupyter    1220308 Dec  8 20:32 trnL_cluster.swarms
-rw-r--r--  1 jupyter jupyter     202860 Dec  8 20:32 trnL_cluster.stats
-rw-r--r--  1 jupyter jupyter     410257 Dec  8 20:32 trnL_cluster_representatives2blast.fas
-rw-r--r--  1 jupyter jupyter     433105 Dec  8 20:32 trnL_cluster_representatives.fas
-rw-r--r--  1 jupyter jupyter    2786227 Dec  8 20:32 trnL_global_dp.fas
drwxr-xr-x  8 jupyter jupyter       4096 Dec  8 20:31 .git/


In [54]:
!head -4 trnL_cluster_representatives.fas

>a84c749701d67d1226018c6714b72e543a618f16;size=64940;
CTTTTTTTATATAAAAAAAAATTTTATAATATAATAAAAAATTTAGAATAATAATATTCTATATATATTATATATTTTTTTATATTTTTTCAAAATAATGAT
>ac90db76fbae68003ec939d36b9847a46cbfa3c7;size=51129;
ATCCTCTTCTCTTTTCCAAGAACAAACAGGGGTTCAGAAAGCGAAAAAGGGG


In [55]:
!head -4 trnL_cluster_representatives2blast.fas

>a84c749701d67d1226018c6714b72e543a618f16_64940
CTTTTTTTATATAAAAAAAAATTTTATAATATAATAAAAAATTTAGAATAATAATATTCTATATATATTATATATTTTTTTATATTTTTTCAAAATAATGAT
>ac90db76fbae68003ec939d36b9847a46cbfa3c7_51129
ATCCTCTTCTCTTTTCCAAGAACAAACAGGGGTTCAGAAAGCGAAAAAGGGG


In [56]:
!head -4 trnL_cluster.stats

2684	64940	a84c749701d67d1226018c6714b72e543a618f16	44856	1949	8	8
989	51129	ac90db76fbae68003ec939d36b9847a46cbfa3c7	43740	730	5	5
865	43673	2987db85c9fae5a1464f814e070b7507621e8a57	36626	635	4	4
767	37670	a069993ab97b3f720cf66567bfb1fad838b20dc6	32359	553	5	5


In [57]:
!head -4 trnL_cluster_representatives.uchime

0.0000	a84c749701d67d1226018c6714b72e543a618f16;size=64940;	*	*	*	*	*	*	*	*	0	0	0	0	0	0	*	N
0.0000	ac90db76fbae68003ec939d36b9847a46cbfa3c7;size=51129;	*	*	*	*	*	*	*	*	0	0	0	0	0	0	*	N
0.0000	2987db85c9fae5a1464f814e070b7507621e8a57;size=43673;	*	*	*	*	*	*	*	*	0	0	0	0	0	0	*	N
0.0000	addff39eca064dd9caa294721e6a23cab9c331ab;size=41429;	*	*	*	*	*	*	*	*	0	0	0	0	0	0	*	N


In [58]:
!bash 5_BLAST_options.sh -h

##################################################
BLAST alignment against custom or NCBI NT local database.
 
Usage: 5_BLAST_options.sh [-f fasta] [-d database] [-s float] [-c float] [-t treads] [-o output]
-f     representative OTUs multifasta from clustering step
-d     database to align against: 'NT' or 'My_refs.uniq.fasta'
-s     minimum similarity between query and subject, float 0-1
-c     minimum subject coverage reached by the query alignment, float 0-1
-t     number of threads:
           custom database: 2 threads for each 1000 seqs.
           NCBI NT database: 4 threads for each 1000 seqs.
-o     output name: 'name'_local_blast.tab
-h     print this help
 
##################################################


In [59]:
%%bash
./5_BLAST_options.sh -f trnL_cluster_representatives2blast.fas -d target_trnL.uniq.fasta -s 0.95 -c 0.5 -t 4 -o target95_trnL



Building a new DB, current time: 12/08/2021 20:34:50
New DB name:   /home/jupyter/METAPIPE-Notebooks/target_trnL.uniq.fasta
New DB title:  target_trnL.uniq.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 15359 sequences in 0.618003 seconds.




./5_BLAST_options.sh: 59: [: target_trnL.uniq.fasta: unexpected operator
./5_BLAST_options.sh: 59: [: target_trnL.uniq.fasta: unexpected operator


In [ ]:
!update_blastdb.pl --decompress nr

In [60]:
%ls -lat | head

total 3935556
drwxr-xr-x 10 jupyter jupyter     106496 Dec  8 20:34 ./
-rw-r--r--  1 jupyter jupyter      99741 Dec  8 20:34 target95_trnL_OTU_tax_assignments.txt
-rw-r--r--  1 jupyter jupyter     118761 Dec  8 20:34 Blast_results_formatted2OTUtable.tab
-rw-r--r--  1 jupyter jupyter     121934 Dec  8 20:34 target95_trnL_local_blast.tab
-rw-r--r--  1 jupyter jupyter     749568 Dec  8 20:34 target_trnL.uniq.fasta.ndb
-rw-r--r--  1 jupyter jupyter     291829 Dec  8 20:34 target_trnL.uniq.fasta.nos
-rw-r--r--  1 jupyter jupyter      16384 Dec  8 20:34 target_trnL.uniq.fasta.ntf
-rw-r--r--  1 jupyter jupyter      61440 Dec  8 20:34 target_trnL.uniq.fasta.nto
-rw-r--r--  1 jupyter jupyter     184316 Dec  8 20:34 target_trnL.uniq.fasta.not


In [61]:
!head target95_trnL_local_blast.tab

ac90db76fbae68003ec939d36b9847a46cbfa3c7_51129	100.000	Eukaryota|Streptophyta|Magnoliopsida|Sapindales|Rutaceae|Citrus|Citrus+micrantha	gb|KY656130.1|	N/A
2987db85c9fae5a1464f814e070b7507621e8a57_43673	100.000	Eukaryota|Streptophyta|Magnoliopsida|Poales|Poaceae|Zea|Zea+mays	gb|MN715130.1|	N/A
addff39eca064dd9caa294721e6a23cab9c331ab_41429	100.000	Eukaryota|Streptophyta|Magnoliopsida|Apiales|Apiaceae|Selinum|Selinum+carvifolia	gb|MT846883.1|	N/A
a069993ab97b3f720cf66567bfb1fad838b20dc6_37670	100.000	Eukaryota|Streptophyta|Magnoliopsida|Laurales|Lauraceae|Cinnamomum|Cinnamomum+camphora	gb|MN482111.1|	N/A
28bf85082548d49c8d3b2793064a2fae0f417562_34741	100.000	Eukaryota|Streptophyta|Magnoliopsida|Rosales|Rosaceae|Prunus|Prunus+sunhangii	gb|MK411811.1|	N/A
967e951648145482fcc47f85b2a3530bec26d552_33003	100.000	Eukaryota|Streptophyta|Magnoliopsida|Apiales|Apiaceae|Selinum|Selinum+carvifolia	gb|MT846883.1|	N/A
36ecc3a6b7c304e724eb2680f97c751acaa0cdf6_20092	100.000	Eukaryota|Streptophyta|Magno

In [62]:
%ls -t | head

target95_trnL_OTU_tax_assignments.txt
Blast_results_formatted2OTUtable.tab
target95_trnL_local_blast.tab
target_trnL.uniq.fasta.ndb
target_trnL.uniq.fasta.nos
target_trnL.uniq.fasta.ntf
target_trnL.uniq.fasta.nto
target_trnL.uniq.fasta.not
target_trnL.uniq.fasta.nhr
target_trnL.uniq.fasta.nin


In [63]:
!head Blast_results_formatted2OTUtable.tab

ac90db76fbae68003ec939d36b9847a46cbfa3c7	51129	100.000	Eukaryota|Streptophyta|Magnoliopsida|Sapindales|Rutaceae|Citrus|Citrus+micrantha	KY656130.1	N/A
2987db85c9fae5a1464f814e070b7507621e8a57	43673	100.000	Eukaryota|Streptophyta|Magnoliopsida|Poales|Poaceae|Zea|Zea+mays	MN715130.1	N/A
addff39eca064dd9caa294721e6a23cab9c331ab	41429	100.000	Eukaryota|Streptophyta|Magnoliopsida|Apiales|Apiaceae|Selinum|Selinum+carvifolia	MT846883.1	N/A
a069993ab97b3f720cf66567bfb1fad838b20dc6	37670	100.000	Eukaryota|Streptophyta|Magnoliopsida|Laurales|Lauraceae|Cinnamomum|Cinnamomum+camphora	MN482111.1	N/A
28bf85082548d49c8d3b2793064a2fae0f417562	34741	100.000	Eukaryota|Streptophyta|Magnoliopsida|Rosales|Rosaceae|Prunus|Prunus+sunhangii	MK411811.1	N/A
967e951648145482fcc47f85b2a3530bec26d552	33003	100.000	Eukaryota|Streptophyta|Magnoliopsida|Apiales|Apiaceae|Selinum|Selinum+carvifolia	MT846883.1	N/A
36ecc3a6b7c304e724eb2680f97c751acaa0cdf6	20092	100.000	Eukaryota|Streptophyta|Magnoliopsida|Fabales|Fabacea

In [64]:
!head target95_trnL_OTU_tax_assignments.txt

amplicon	Kingdom	Phylum	Class	Ord	Family	Genus	Species	SIM	OTU_abundance
ac90db76fbae68003ec939d36b9847a46cbfa3c7	Eukaryota	Streptophyta	Magnoliopsida	Sapindales	Rutaceae	Citrus	micrantha	100.000	51129
2987db85c9fae5a1464f814e070b7507621e8a57	Eukaryota	Streptophyta	Magnoliopsida	Poales	Poaceae	Zea	mays	100.000	43673
addff39eca064dd9caa294721e6a23cab9c331ab	Eukaryota	Streptophyta	Magnoliopsida	Apiales	Apiaceae	Selinum	carvifolia	100.000	41429
a069993ab97b3f720cf66567bfb1fad838b20dc6	Eukaryota	Streptophyta	Magnoliopsida	Laurales	Lauraceae	Cinnamomum	camphora	100.000	37670
28bf85082548d49c8d3b2793064a2fae0f417562	Eukaryota	Streptophyta	Magnoliopsida	Rosales	Rosaceae	Prunus	sunhangii	100.000	34741
967e951648145482fcc47f85b2a3530bec26d552	Eukaryota	Streptophyta	Magnoliopsida	Apiales	Apiaceae	Selinum	carvifolia	100.000	33003
36ecc3a6b7c304e724eb2680f97c751acaa0cdf6	Eukaryota	Streptophyta	Magnoliopsida	Fabales	Fabaceae	Glycyrrhiza	glabra	100.000	20092
7bb64ef2725b146c74add357867e20995ecd52ee	

In [65]:
 %%bash
./6_build_OTU_table.sh trnL_cluster_representatives.fas trnL_sub.qual Blast_results_formatted2OTUtable.tab \
/home/jupyter/METAPIPE-Notebooks/dereplicated

In [66]:
%ls -t | head

trnL_cluster_OTU_table_complete.tab
metapipe-run.ipynb*
target95_trnL_OTU_tax_assignments.txt
Blast_results_formatted2OTUtable.tab
target95_trnL_local_blast.tab
target_trnL.uniq.fasta.ndb
target_trnL.uniq.fasta.nos
target_trnL.uniq.fasta.ntf
target_trnL.uniq.fasta.nto
target_trnL.uniq.fasta.not


In [67]:
!head -3 trnL_cluster_OTU_table_complete.tab

OTU	total	cloud	amplicon	length	abundance	chimera	spread	quality	sequence	identity	taxonomy	references	Angelicaesinensis_a_dp	Angelicaesinensis_b_dp	Carthamustinctorius_a_dp	Carthamustinctorius_b_dp	Citrusaurantium_a_dp	Citrusaurantium_b_dp	Cyperusrotundus_a_dp	Cyperusrotundus_b_dp	GXZYT_a_dp	GXZYT_b_dp	GZFLW_a_dp	GZFLW_b_dp	Glycyrrhizauralensis_a_dp	Glycyrrhizauralensis_b_dp	GuiZhiFuLingPian_a_dp	GuiZhiFuLingPian_b_dp	GuiZhiFuLingTan_a_dp	GuiZhiFuLingTan_b_dp	Ligusticumchuanxiong_a_dp	Ligusticumchuanxiong_b_dp	Linderaaggregata_a_dp	Linderaaggregata_b_dp	NegativePCR_dp	Negative_dp	Paeonialactiflora_a_dp	Paeonialactiflora_b_dp	Peoniasuffruticosa_a_dp	Peoniasuffruticosa_b_dp	Poriacocos_a_dp	Poriacocos_b_dp	Prunuspersica_a_dp	Prunuspersica_b_dp	Trogopterusxanthipes_a_dp	Trogopterusxanthipes_b_dp
1	64940	2684	a84c749701d67d1226018c6714b72e543a618f16	102	44856	N	12	0.0001	CTTTTTTTATATAAAAAAAAATTTTATAATATAATAAAAAATTTAGAATAATAATATTCTATATATATTATATATTTTTTTATATTTTTTCAAAATAATGAT	NA	NA	NA	0	18551	

In [68]:
%%bash 
./6_filter_OTU_table.sh trnL_cluster_OTU_table_complete.tab trnL_sub.qual

In [69]:
%ls -t | head -20

trnL_sub.ee
trnL_cluster_OTU_table_f4_no_hit2_cut2phyloseq2.tab
trnL_cluster_OTU_table_f4_no_hit2.tab
trnL_cluster_OTU_table_f3_hits_only2_cut2phyloseq2.tab
trnL_cluster_OTU_table_f3_hits_only2.tab
trnL_cluster_OTU_table_f2_97up2_cut2phyloseq2.tab
trnL_cluster_OTU_table_f2_97up2.tab
trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab
trnL_cluster_OTU_table_f1_basics2.tab
trnL_cluster_OTU_table_complete.tab
metapipe-run.ipynb*
target95_trnL_OTU_tax_assignments.txt
Blast_results_formatted2OTUtable.tab
target95_trnL_local_blast.tab
target_trnL.uniq.fasta.ndb
target_trnL.uniq.fasta.nos
target_trnL.uniq.fasta.ntf
target_trnL.uniq.fasta.nto
target_trnL.uniq.fasta.not
target_trnL.uniq.fasta.nhr


In [70]:
!head -5 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b
a84c749701d67d1226018c6714b72e543a618f16	0	18551	25637	12	0	7	0	0	0	0	0	12	13	66	0	0	23	0	0	4	0	0	0	0	0	0	0	0	0	0	0	1	15	20599
ac90db76fbae68003ec939d36b9847a46cbfa3c7	0	0	1	29	171	49425	32	137	828	13	0	0	4	0	19	8	2	10	6	1	0	0	6	121	1	4	0	0	1	1	2	16	290	1
2987db85c9fae5a1464f814e070b7507621e8a57	23	0	1	1	4	2	0	0	0	0	40910	1	1	0	2584	2	0	75	4	0	2	1	4	1	3	0	3	0	0	0	1	3	45	2
addff39eca064dd9c

In [71]:
!head trnL_sub_metadata.txt

SampleID	SampleName	Replicate	Experiment	Exp_fam	Exp_genus
Angelicaesinensis_a	Angelicaesinensis_a	Angelicaesinensis	Exp1	Apiaceae	Angelica
Angelicaesinensis_b	Angelicaesinensis_b	Angelicaesinensis	Exp1	Apiaceae	Angelica
Carthamustinctorius_a	Carthamustinctorius_a	Carthamustinctorius	Exp2	Asteraceae	Carthamus
Carthamustinctorius_b	Carthamustinctorius_b	Carthamustinctorius	Exp2	Asteraceae	Carthamus
Citrusaurantium_a	Citrusaurantium_a	Citrusaurantium	Exp3	Rutaceae	Citrus
Citrusaurantium_b	Citrusaurantium_b	Citrusaurantium	Exp3	Rutaceae	Citrus
Cyperusrotundus_a	Cyperusrotundus_a	Cyperusrotundus	Exp4	Cypereae	Cyperus
Cyperusrotundus_b	Cyperusrotundus_b	Cyperusrotundus	Exp4	Cypereae	Cyperus
Glycyrrhizauralensis_a	Glycyrrhizauralensis_a	Glycyrrhizauralensis	Exp5	Fabaceae	Glycyrrhiza


In [72]:
!gsutil cp trnL_sub_metadata.txt gs://metapipe-files

Copying file://trnL_sub_metadata.txt [Content-Type=text/plain]...
/ [1 files][  2.6 KiB/  2.6 KiB]                                                
Operation completed over 1 objects/2.6 KiB.                                      


In [73]:
!gsutil cp trnL_cluster_OTU_table_f* gs://metapipe-files

Copying file://trnL_cluster_OTU_table_f1_basics2.tab [Content-Type=application/octet-stream]...
Copying file://trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab [Content-Type=application/octet-stream]...
Copying file://trnL_cluster_OTU_table_f2_97up2.tab [Content-Type=application/octet-stream]...
Copying file://trnL_cluster_OTU_table_f2_97up2_cut2phyloseq2.tab [Content-Type=application/octet-stream]...
\ [4 files][213.0 KiB/213.0 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://trnL_cluster_OTU_table_f3_hits_only2.tab [Content-Type=application/octet-stream]...
Copying file://trnL_cluster_OTU_table_f3_hits_only2_cut2phyloseq2.tab [Content-Type=application/octet-stream]...
Copying file://trnL_cluster_OTU_table_f4_no_hit

https://console.cloud.google.com/marketplace/product/rstudio-launcher-public/rstudio-server-pro-standard-for-gcp?q=rstudio&id=76ae9647-2d08-4888-b737-74f751581848&project=metapipe

In [76]:
!gsutil cp target95_trnL_OTU_tax_assignments.txt gs://metapipe-files

Copying file://target95_trnL_OTU_tax_assignments.txt [Content-Type=text/plain]...
- [1 files][ 97.4 KiB/ 97.4 KiB]                                                
Operation completed over 1 objects/97.4 KiB.                                     


In [78]:
!grep "Dioscoreaceae" target95_trnL_OTU_tax_assignments.txt

c65370ec20a9cdf9089e1983d7c5c3d4227a870b	Eukaryota	Streptophyta	Magnoliopsida	Dioscoreales	Dioscoreaceae	Dioscorea	japonica	100.000	14276
94385ff6f775ede0a8b76f5e4f3a9a0de58a4eca	Eukaryota	Streptophyta	Magnoliopsida	Dioscoreales	Dioscoreaceae	Dioscorea	japonica	100.000	7332
bd872bc5ee72741e44de0543feee4dd24b70ef14	Eukaryota	Streptophyta	Magnoliopsida	Dioscoreales	Dioscoreaceae	Dioscorea	japonica	100.000	4
15d9365c21468a243e67bbe6a87f3ccd935e8c79	Eukaryota	Streptophyta	Magnoliopsida	Dioscoreales	Dioscoreaceae	Dioscorea	japonica	95.960	2
2033a8682dea95436c0e460e023c755612c5de9a	Eukaryota	Streptophyta	Magnoliopsida	Dioscoreales	Dioscoreaceae	Dioscorea	japonica	100.000	2
6c7ed8e20fcd9e7459f65fdd101b460439443287	Eukaryota	Streptophyta	Magnoliopsida	Dioscoreales	Dioscoreaceae	Dioscorea	matsudae	94.737	2
01274e831ee4e9767101786b61b716fb2a5b8488	Eukaryota	Streptophyta	Magnoliopsida	Dioscoreales	Dioscoreaceae	Dioscorea	japonica	98.990	1
088b86d5a292b92fd5e89b4ff2cb82d852498f60	Eukaryota	Strepto

Gostaria de fazer um barplotzinho para mostrar isso...

Toda essa parte a seguir deveria mostrar os plots...

Check the Top6 OTUs from the whole dataset:

In [86]:
!grep "ac90db76fbae68003ec939d36b9847a46cbfa3c7" target95_trnL_OTU_tax_assignments.txt

ac90db76fbae68003ec939d36b9847a46cbfa3c7	Eukaryota	Streptophyta	Magnoliopsida	Sapindales	Rutaceae	Citrus	micrantha	100.000	51129


In [100]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [101]:
!grep "ac90db76fbae68003ec939d36b9847a46cbfa3c7" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

ac90db76fbae68003ec939d36b9847a46cbfa3c7	0	0	1	29	171	49425	32	137	828	13	0	0	4	0	19	8	2	10	6	1	0	0	6	121	1	4	0	0	1	1	2	16	290	1


We observe that Rutaceae Citrus micrantha is more abundant in Citrusaurantium_b sample.

In [87]:
!grep "2987db85c9fae5a1464f814e070b7507621e8a57" target95_trnL_OTU_tax_assignments.txt

2987db85c9fae5a1464f814e070b7507621e8a57	Eukaryota	Streptophyta	Magnoliopsida	Poales	Poaceae	Zea	mays	100.000	43673


In [102]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [103]:
!grep "2987db85c9fae5a1464f814e070b7507621e8a57" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

2987db85c9fae5a1464f814e070b7507621e8a57	23	0	1	1	4	2	0	0	0	0	40910	1	1	0	2584	2	0	75	4	0	2	1	4	1	3	0	3	0	0	0	1	3	45	2


We observe that Poaceae Zea mays is more abundant in GZFLW_a sample.

In [88]:
!grep "addff39eca064dd9caa294721e6a23cab9c331ab" target95_trnL_OTU_tax_assignments.txt

addff39eca064dd9caa294721e6a23cab9c331ab	Eukaryota	Streptophyta	Magnoliopsida	Apiales	Apiaceae	Selinum	carvifolia	100.000	41429


In [104]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [105]:
!grep "addff39eca064dd9caa294721e6a23cab9c331ab" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

addff39eca064dd9caa294721e6a23cab9c331ab	18563	9615	5	6	3	62	36	4	12103	143	16	11	0	17	123	1	9	80	40	64	7	2	75	3	23	65	4	5	27	3	6	203	97	8


We observe that Apiaceae Selinum carvifolia is more abundant in Angelicaesinensis_a sample.

In [89]:
!grep "a069993ab97b3f720cf66567bfb1fad838b20dc6" target95_trnL_OTU_tax_assignments.txt

a069993ab97b3f720cf66567bfb1fad838b20dc6	Eukaryota	Streptophyta	Magnoliopsida	Laurales	Lauraceae	Cinnamomum	camphora	100.000	37670


In [106]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [107]:
!grep "a069993ab97b3f720cf66567bfb1fad838b20dc6" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

a069993ab97b3f720cf66567bfb1fad838b20dc6	0	2	0	0	0	5	4480	15	0	0	14	0	0	0	33027	1	0	116	0	0	0	0	0	0	0	0	0	0	4	0	0	5	1	0


We observe that Lauraceae Cinnamomum camphora is more abundant in Cyperusrotundus_a sample.

In [90]:
!grep "28bf85082548d49c8d3b2793064a2fae0f417562" target95_trnL_OTU_tax_assignments.txt

28bf85082548d49c8d3b2793064a2fae0f417562	Eukaryota	Streptophyta	Magnoliopsida	Rosales	Rosaceae	Prunus	sunhangii	100.000	34741


In [108]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [109]:
!grep "28bf85082548d49c8d3b2793064a2fae0f417562" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

28bf85082548d49c8d3b2793064a2fae0f417562	14	20	1	20	3	32	9	2	5763	167	5260	1	1	0	18	15	5	962	6	34	0	1	26	10	103	32	41	33	32	9	9954	12102	64	1


We observe that Rosaceae Prunus sunhangii is more abundant in Prunuspersica_b sample.

In [91]:
!grep "967e951648145482fcc47f85b2a3530bec26d552" target95_trnL_OTU_tax_assignments.txt

967e951648145482fcc47f85b2a3530bec26d552	Eukaryota	Streptophyta	Magnoliopsida	Apiales	Apiaceae	Selinum	carvifolia	100.000	33003


In [110]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [111]:
!grep "967e951648145482fcc47f85b2a3530bec26d552" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

967e951648145482fcc47f85b2a3530bec26d552	294	4	3	23	21	329	2	12	0	0	12723	0	9	5	754	32	21	13889	16	66	64	0	108	40	0	12	4110	70	2	11	10	266	2	105


We observe that Apiaceae Selinum carvifolia is more abundant in GuiZhiFuLingTan_b sample.


Check the top4 OTUs from data_clean dataset

In [82]:
!grep "9319dccb21d181d5ff01c4d084ace1f2bf1d9f87" target95_trnL_OTU_tax_assignments.txt

9319dccb21d181d5ff01c4d084ace1f2bf1d9f87	Eukaryota	Streptophyta	Magnoliopsida	Poales	Poaceae	Campeiostachys	calcicola	100.000	11136


In [99]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [98]:
!grep "9319dccb21d181d5ff01c4d084ace1f2bf1d9f87" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

9319dccb21d181d5ff01c4d084ace1f2bf1d9f87	0	0	1	9	0	0	746	0	0	0	0	6	0	202	0	8	10152	4	0	0	0	0	0	0	0	0	0	0	2	0	0	0	0	6


We observe that Poaceae Campeiostachys calcicola is more abundant in GuiZhiFuLingTan_a sample.

In [83]:
!grep "d9e0449e3f64c10837e21c2f3fb744969044482d" target95_trnL_OTU_tax_assignments.txt

d9e0449e3f64c10837e21c2f3fb744969044482d	Eukaryota	Streptophyta	Magnoliopsida	Poales	Poaceae	Campeiostachys	calcicola	100.000	13459


In [97]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [96]:
!grep "d9e0449e3f64c10837e21c2f3fb744969044482d" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

d9e0449e3f64c10837e21c2f3fb744969044482d	1	0	0	1	0	1	12869	38	0	0	0	0	3	0	0	1	524	1	0	0	1	0	0	0	0	0	1	0	4	0	0	0	14	0


We observe that Poaceae Campeiostachys calcicola is also more abundant in Cyperusrotundus_a sample.

In [84]:
!grep "785131f35679fb33afaedf6919f8ec30539638bc" target95_trnL_OTU_tax_assignments.txt

785131f35679fb33afaedf6919f8ec30539638bc	Eukaryota	Streptophyta	Pinopsida	NA	Cupressaceae	Thuja	sp.	100.000	13999


In [95]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [94]:
!grep "785131f35679fb33afaedf6919f8ec30539638bc" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

785131f35679fb33afaedf6919f8ec30539638bc	1	1	0	0	0	3	0	0	18	0	8	0	0	1	0	30	3239	64	0	0	0	0	0	0	0	0	0	0	8	1	0	0	10625	0


We observe that Cupressaceae Thuja sp. is more abundant in Trogopterusxanthipes_a sample.

In [112]:
!grep "a069993ab97b3f720cf66567bfb1fad838b20dc6" target95_trnL_OTU_tax_assignments.txt

a069993ab97b3f720cf66567bfb1fad838b20dc6	Eukaryota	Streptophyta	Magnoliopsida	Laurales	Lauraceae	Cinnamomum	camphora	100.000	37670


In [113]:
!head -1 trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

amplicon	Angelicaesinensis_a	Angelicaesinensis_b	Carthamustinctorius_a	Carthamustinctorius_b	Citrusaurantium_a	Citrusaurantium_b	Cyperusrotundus_a	Cyperusrotundus_b	GXZYT_a	GXZYT_b	GZFLW_a	GZFLW_b	Glycyrrhizauralensis_a	Glycyrrhizauralensis_b	GuiZhiFuLingPian_a	GuiZhiFuLingPian_b	GuiZhiFuLingTan_a	GuiZhiFuLingTan_b	Ligusticumchuanxiong_a	Ligusticumchuanxiong_b	Linderaaggregata_a	Linderaaggregata_b	NegativePCR	Negative	Paeonialactiflora_a	Paeonialactiflora_b	Peoniasuffruticosa_a	Peoniasuffruticosa_b	Poriacocos_a	Poriacocos_b	Prunuspersica_a	Prunuspersica_b	Trogopterusxanthipes_a	Trogopterusxanthipes_b


In [114]:
!grep "a069993ab97b3f720cf66567bfb1fad838b20dc6" trnL_cluster_OTU_table_f1_basics2_cut2phyloseq2.tab

a069993ab97b3f720cf66567bfb1fad838b20dc6	0	2	0	0	0	5	4480	15	0	0	14	0	0	0	33027	1	0	116	0	0	0	0	0	0	0	0	0	0	4	0	0	5	1	0


We observe that Lauraceae Cinnamomum camphorais more abundant in GuiZhiFuLingPian_a sample